# Data Wrangling

## Spark Loading

In [1]:
%load_ext sparkmagic.magics

In [2]:
import os
from IPython import get_ipython
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-homework3", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [3]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-homework3 -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5397,application_1649870447656_2016,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [4]:
%%spark
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.3.2.3.1.4.0-315

## Stops 15km from Zürich HB (8503000) (lat, lon) = (47.378177, 8.540192)

In [5]:
%%spark
all_stops = spark.read.csv('/data/sbb/csv/allstops/stop_locations.csv')
all_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+----+----+
|    _c0|                 _c1|             _c2|             _c3| _c4| _c5|
+-------+--------------------+----------------+----------------+----+----+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|null|null|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|null|null|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|null|null|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|null|null|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|null|null|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|null|null|
|1100013|  Mambach, Silbersau|47.7395192233867|7.88223152899259|null|null|
|1100014|Fröhnd (Schwarzw)...|47.7543663509316|7.88913059037559|null|null|
|1100015|Fröhnd (Schwarzw)...|47.7605926689054|7.88553732923861|null|null|
|1100016|     Wembach (Baden)|47.7728317637339|7.88772023537933|null|null|
+-------+----------------

In [6]:
%%spark
all_stops = all_stops.withColumnRenamed('_c0','stop_id')
all_stops = all_stops.withColumnRenamed('_c1','stop_name')
all_stops = all_stops.withColumnRenamed('_c2','lat')
all_stops = all_stops.withColumnRenamed('_c3','lon')
all_stops = all_stops.drop('_c4','_c5')
all_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+
|stop_id|           stop_name|             lat|             lon|
+-------+--------------------+----------------+----------------+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|
|1100013|  Mambach, Silbersau|47.7395192233867|7.88223152899259|
|1100014|Fröhnd (Schwarzw)...|47.7543663509316|7.88913059037559|
|1100015|Fröhnd (Schwarzw)...|47.7605926689054|7.88553732923861|
|1100016|     Wembach (Baden)|47.7728317637339|7.88772023537933|
+-------+--------------------+----------------+----------------+
only showing top 10 rows

In [7]:
%%spark
import pyspark.sql.functions as F
zur_hp_lat = 47.378177
zur_hp_lon = 8.540192

all_stops = all_stops.withColumn("a", (
        F.pow(F.sin(F.radians(zur_hp_lat - F.col("lat")) / 2), 2) +
        F.cos(F.radians(F.col("lat"))) * F.cos(F.radians(F.lit(zur_hp_lat))) *
        F.pow(F.sin(F.radians(zur_hp_lon - F.col("lon")) / 2), 2)
    )).withColumn("distance", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)

all_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+--------------------+------------------+
|stop_id|           stop_name|             lat|             lon|                   a|          distance|
+-------+--------------------+----------------+----------------+--------------------+------------------+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|2.475398503604094...|  63396.0142787592|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|2.445985737384344...| 63018.24892966985|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|2.446306465224431...| 63022.38043559904|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|2.409494241610368E-5|62546.396144882805|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|2.456510835903595E-5| 63153.68854022285|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|2.469960438154199...| 63326.33982721011|
|1100013|  Mambach, Silbersau|47.7395192233867|7.882231

In [8]:
%%spark
close_stops = all_stops.filter(all_stops['distance'] <= 15000).drop('a','distance')
close_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+----------------+----------------+
|      stop_id|           stop_name|             lat|             lon|
+-------------+--------------------+----------------+----------------+
|          176|Zimmerberg-Basist...|47.3516780901371|8.52195777551452|
|      8500926|Oetwil a.d.L., Sc...|47.4236270123012| 8.4031825286317|
|      8502075|Zürich Flughafen,...|47.4510244676285|8.56372943623189|
|      8502186|Dietikon Stoffelbach|47.3933267759652|8.39896044679575|
|    8502186:0|Dietikon Stoffelbach|47.3933997509195|8.39894248049007|
|8502186:0:1/2|Dietikon Stoffelbach|47.3933997509195|8.39894248049007|
|     8502186P|Dietikon Stoffelbach|47.3933997509195|8.39894248049007|
|      8502187|Rudolfstetten Hof...|47.3646702178563|8.37695172233176|
|    8502187:0|Rudolfstetten Hof...|47.3647371479356|8.37703257070734|
|8502187:0:1/2|Rudolfstetten Hof...|47.3647371479356|8.37703257070734|
+-------------+--------------------+----------------+----------------+
only s

In [9]:
%%spark
close_stops.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2122

## Weekly timetable

Considered period: May 13-17, 2019

In [10]:
%%spark
all_trips1 = spark.read.csv('/data/sbb/csv/stop_times/2019/05/08/stop_times.txt', header=True)
all_trips2 = spark.read.csv('/data/sbb/csv/stop_times/2019/05/15/stop_times.txt', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
%%spark
all_trips1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10862563

In [12]:
%%spark
all_trips2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11128930

In [13]:
%%spark
all_trips_tot = all_trips1.union(all_trips2)
all_trips_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------+--------------+-------+-------------+-----------+-------------+
|            trip_id|arrival_time|departure_time|stop_id|stop_sequence|pickup_type|drop_off_type|
+-------------------+------------+--------------+-------+-------------+-----------+-------------+
|1.TA.1-84-j19-1.1.H|    06:13:00|      06:13:00|8572249|            1|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:14:00|      06:14:00|8577508|            2|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:15:00|      06:15:00|8581070|            3|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:16:00|      06:16:00|8578360|            4|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:17:00|      06:17:00|8583448|            5|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:18:00|      06:19:00|8578359|            6|          0|            0|
|1.TA.1-84-j19-1.1.H|    06:24:00|      06:24:00|8578358|            7|          0|            0|
|1.TA.1-84-j19-1.1.H

In [14]:
%%spark
routes1 = spark.read.csv('/data/sbb/csv/trips/2019/05/08/trips.txt', header=True)
routes2 = spark.read.csv('/data/sbb/csv/trips/2019/05/15/trips.txt', header=True)
routes_tot = routes1.union(routes2)
routes_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+-------------------+---------------+------------+
|   route_id|service_id|             trip_id|      trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+-------------------+---------------+------------+
| 1-85-j19-1|  TA+b0001| 2.TA.1-85-j19-1.1.H|Schöftland, Bahnhof|          85003|           0|
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            108|           1|
|1-1-C-j19-1|  TA+b0001|7.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            112|           1|
|1-1-C-j19-1|  TA+b0001|9.TA.1-1-C-j19-1.3.R| Zofingen, Altachen|            116|           1|
|1-1-C-j19-1|  TA+b0001|11.TA.1-1-C-j19-1...| Zofingen, Altachen|            120|           1|
|1-1-C-j19-1|  TA+b0001|13.TA.1-1-C-j19-1...| Zofingen, Altachen|            124|           1|
|1-1-C-j19-1|  TA+b0001|15.TA.1-1-C-j19-1...| Zofingen, Altachen|            128|           1|
|1-1-C-j19-1|  TA+b0001|17.TA.1-1-C-j19-1...| Zofi

In [15]:
%%spark
trips_with_routes = all_trips_tot.join(routes_tot, all_trips_tot.trip_id == routes_tot.trip_id)
trips_with_routes.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+------------+-------------+-----------+-------------+-------------+----------+--------------------+--------------------+---------------+------------+
|             trip_id|arrival_time|departure_time|     stop_id|stop_sequence|pickup_type|drop_off_type|     route_id|service_id|             trip_id|       trip_headsign|trip_short_name|direction_id|
+--------------------+------------+--------------+------------+-------------+-----------+-------------+-------------+----------+--------------------+--------------------+---------------+------------+
|1.TA.11-12-B-j19-...|    08:30:00|      08:30:00|     8582915|            1|          0|            0|11-12-B-j19-1|  TA+b0deu|1.TA.11-12-B-j19-...|Flims Dorf, Bergb...|            202|           1|
|1.TA.11-12-B-j19-...|    08:31:00|      08:31:00|     8582916|            2|          0|            0|11-12-B-j19-1|  TA+b0deu|1.TA.11-12-B-j19-...|Flims Dorf, Bergb...|            202|           1|


In [16]:
%%spark
trips_with_routes = trips_with_routes.select(all_trips_tot.trip_id, 'service_id').distinct()
trips_with_routes.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|             trip_id|service_id|
+--------------------+----------+
|1.TA.11-12-B-j19-...|  TA+b0deu|
|1.TA.12-1-A-j19-1...|  TA+b099q|
|1.TA.12-1-A-j19-1...|  TA+b08ff|
|1.TA.16-440-j19-1...|  TA+b0007|
|1.TA.16-440-j19-1...|  TA+b0ei3|
|1.TA.23-792-j19-1...|  TA+b000o|
|1.TA.26-18-j19-1.1.H|  TA+b06tm|
|1.TA.26-18-j19-1.1.H|  TA+b0b46|
|1.TA.26-833-j19-1...|  TA+b002d|
|1.TA.26-853-j19-1...|  TA+b001t|
+--------------------+----------+
only showing top 10 rows

In [17]:
%%spark
cal1 = spark.read.csv('/data/sbb/csv/calendar/2019/05/08/calendar.txt', header=True)
cal2 = spark.read.csv('/data/sbb/csv/calendar/2019/05/15/calendar.txt', header=True)
cal_tot = cal1.union(cal2).distinct()
cal_tot.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|start_date|end_date|
+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|  TA+b06hv|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0036|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0ha1|     0|      0|        0|       0|     0|       1|     0|  20181209|20191214|
|  TA+b0nha|     1|      1|        1|       1|     0|       0|     1|  20181209|20191214|
|  TA+b022q|     1|      1|        1|       1|     0|       0|     0|  20181209|20191214|
|  TA+b025m|     0|      0|        0|       0|     1|       0|     0|  20181209|20191214|
|  TA+b0ms4|     0|      0|        0|       0|     0|       0|     1|  20181209|20191214|
|  TA+b0nvl|     0|      0|        0|       0|     0|       1|     1|  20181209|20191214|
|  TA+b0nz

In [18]:
%%spark
cal_filtered = cal_tot.filter((cal_tot.monday == 1) | (cal_tot.tuesday == 1) | (cal_tot.wednesday == 1) | (cal_tot.thursday == 1) | (cal_tot.friday == 1))
cal_filtered_dates = cal_filtered.filter((cal_tot.start_date <= '20190513') & (cal_tot.end_date >= '20190518'))
cal_filtered_dates.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|start_date|end_date|
+----------+------+-------+---------+--------+------+--------+------+----------+--------+
|  TA+b0nha|     1|      1|        1|       1|     0|       0|     1|  20181209|20191214|
|  TA+b022q|     1|      1|        1|       1|     0|       0|     0|  20181209|20191214|
|  TA+b025m|     0|      0|        0|       0|     1|       0|     0|  20181209|20191214|
|  TA+b0nz0|     0|      0|        0|       0|     1|       1|     0|  20181209|20191214|
|  TA+b0o2i|     1|      0|        0|       1|     0|       0|     0|  20181209|20191214|
|  TA+b0obi|     1|      1|        1|       1|     1|       0|     0|  20181209|20191214|
|  TA+b0obp|     0|      1|        0|       0|     1|       1|     1|  20181209|20191214|
|  TA+b0gtx|     1|      1|        1|       1|     1|       0|     1|  20181209|20191214|
|  TA+b0g2

In [19]:
%%spark
service_ids = cal_filtered_dates.select('service_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
%%spark
service_ids.select('service_id')
trips_in_period = trips_with_routes.join(service_ids, service_ids.service_id == trips_with_routes.service_id,'inner').select('trip_id').distinct()
trips_in_period.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|             trip_id|
+--------------------+
|1.TA.11-12-B-j19-...|
|1.TA.12-1-A-j19-1...|
|1.TA.16-440-j19-1...|
|1.TA.23-792-j19-1...|
|1.TA.26-18-j19-1.1.H|
|1.TA.26-833-j19-1...|
|1.TA.30-32-Y-j19-...|
|1.TA.6-20-B-j19-1...|
|1.TA.6-M13-j19-1.1.H|
|1.TA.61-211-Y-j19...|
+--------------------+
only showing top 10 rows

In [24]:
%%spark
trips_in_period_list = trips_in_period.select('trip_id').distinct().rdd.flatMap(lambda x: x).collect()
final_trips = all_trips_tot.join(trips_in_period, trips_in_period.trip_id == routes_tot.trip_id).distinct()
final_trips.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1676912

In [26]:
%%spark
all_trips_tot.count()

NameError: name 'all_trips_tot' is not defined

In [ ]:
%%spark
final_trips.count()